In [70]:
import theano
import numpy
import theano.tensor as T
from theano import function
from theano import In

In [1]:
import nltk

In [ ]:
from theano import shared

In [ ]:
from theano import pp

In [ ]:
x,y=T.fscalars('x','y')
#y=T.dscalar('y')
z=x+y
#f=function([x,y],z)
print z.eval({x:1,y:6})
f=function([x,In(y,value=1)],z)

In [ ]:
f(5)

In [ ]:
f(2,3)

In [ ]:
x.type is T.dscalar

In [ ]:
u,v=T.dmatrices('U','V')
#v=T.dmatrix('V')
w=u+v
f1=function([u,v],w)
f1([[1,2],[3,4]],[[1,1],[2,2]])

In [ ]:
w.type

In [ ]:
s=1/(1+T.exp(-x))
t=(1+T.tanh(x/2))/2
f=function([x],[s,t])
f(2)

## Shared variable

In [ ]:
state=shared(0)
print(state.get_value())

In [ ]:
inc=T.iscalar('increment')
acc=function([inc],state,updates=[(state,state+inc)])
acc(3)

In [ ]:
print(state.get_value())

In [ ]:
state.set_value(0)
print(state.get_value())

## Skip shared variable

Following is an example of to skip shared variable

In [ ]:
fn_out=state*2+inc
foo=T.scalar(dtype=state.dtype)
skip_shared=function([inc,foo],fn_out,givens=[(state,foo)],on_unused_input='ignore')
skip_shared(2,5)

## Computing gradient

In [ ]:
x=T.dscalar('x')
y=T.exp(-x)/(1+T.exp(-x))
gy=T.grad(y,x)
pp(gy)
# theano.printing.debugprint(gy)

## Scan: compute polynomial

In [7]:
coefficients = T.vector('coefficients')
x=T.scalar('x')
max_coef=100
components, updates = theano.scan(fn=lambda coefficient,power,freevar: coefficient*(freevar**power),
                                  outputs_info=None,
                                  sequences=[coefficients,T.arange(max_coef)],
                                  non_sequences=x
                                 )
polynomial=components.sum()
calc_poly=function([coefficients,x],polynomial)
test_coef=numpy.asarray([2,4,1],dtype=numpy.float32)
test_val=3
print calc_poly(test_coef,test_val)
print (2.0*(3**0)+4*(3**1)+1*(3**2))

23.0
23.0


In [143]:
x=T.vector('x')
k=T.iscalar('k')
# outputs_info=T.as_tensor_variable(numpy.asarray(0,dtype=x.dtype))
outputs_info=T.ones_like(x)
results,updates = theano.scan(fn=lambda prev,x: prev+1/x,
                              outputs_info=outputs_info,
                              non_sequences=x,
                              n_steps=k)
f=function(inputs=[k,x],outputs=results[-1],updates=updates)
f(3,range(1,5))

array([ 4.  ,  2.5 ,  2.  ,  1.75])

In [ ]:
T.neq([1,2,3,4,5],[7,2,3,4,9]).eval()

In [67]:
x=theano.shared(name='x',value=numpy.asarray([[1,2,3],[10,20,30]]).astype(theano.config.floatX))
w=theano.shared(name='w',value=numpy.asarray([[1,2,3,4],
   [1,2,3,5],
   [1,7,3,4]]).astype(theano.config.floatX))
print x.shape.eval()
print w.shape.eval()
b=theano.shared(name='b',value=numpy.ones(3,dtype=theano.config.floatX))
T.dot(x,w).eval()
T.nnet.sigmoid(T.dot(x,w)).eval()

[2 3]
[3 4]


array([[ 0.99752738,  1.        ,  0.99999998,  1.        ],
       [ 1.        ,  1.        ,  1.        ,  1.        ]])

In [105]:
a=theano.shared(numpy.random.rand(4,5,3))
# print numpy.reshape(a,(3,2,-1))
print a.eval()
a.dimshuffle(1,2,0).eval()
b=theano.shared(numpy.random.rand(4,5))
b.dimshuffle(1,'x',0).eval()

[[[ 0.28829626  0.76316841  0.4144818 ]
  [ 0.13689352  0.38445645  0.41134934]
  [ 0.63894833  0.82534978  0.7540116 ]
  [ 0.10697035  0.49909223  0.83461933]
  [ 0.31361896  0.78747879  0.53378372]]

 [[ 0.09712202  0.83871254  0.97799954]
  [ 0.9674433   0.77115889  0.48363575]
  [ 0.59755713  0.08666709  0.71333459]
  [ 0.66909947  0.61578809  0.75785906]
  [ 0.37631077  0.93831452  0.74956211]]

 [[ 0.68755623  0.44909189  0.95416577]
  [ 0.77343921  0.70491339  0.4773451 ]
  [ 0.86440994  0.66326133  0.65763288]
  [ 0.16410777  0.29604263  0.62964892]
  [ 0.77107758  0.8711491   0.89976546]]

 [[ 0.61352857  0.48497414  0.05859162]
  [ 0.06684549  0.8577607   0.48028602]
  [ 0.01558257  0.05621241  0.17184488]
  [ 0.71377493  0.01853575  0.03554421]
  [ 0.98952761  0.2285276   0.97141284]]]


array([[[ 0.42739907,  0.99462243,  0.00213211,  0.99727844]],

       [[ 0.3521462 ,  0.65991659,  0.35201791,  0.22156737]],

       [[ 0.62052579,  0.65337054,  0.27860873,  0.79256099]],

       [[ 0.22682375,  0.14107607,  0.12705962,  0.90014864]],

       [[ 0.54324146,  0.13915365,  0.09249289,  0.57076875]]])

In [168]:
# Normalize vector
x = theano.shared(value=numpy.asarray([[1,2,3,4,5],[6,7,8,1,2]]).astype(theano.config.floatX))
print x.shape.eval()

print T.sqrt((x**2).sum(axis=1)).dimshuffle(0,'x').shape.eval()
# x = x / T.sqrt((x**2).sum(axis=1)).dimshuffle(0,'x')
# y = theano.shared(value=numpy.ones_like(x)*2)
y = theano.shared(value=numpy.asarray([[2],[3]]).astype(theano.config.floatX))
# print y.shape.eval()
print y.shape.eval()
z=x/y

[2 5]
[2 1]
[2 1]
